# Amazon SageMaker Workshop
### _**Data Preparation**_

---
In this part of the workshop we will prepare the data to later train our churn model.

---

## Contents

1. [Background](#Background) - Getting the rawata prepared in the previous lab.
2. [Prepare](#Prepare) - Prepare the data with [Amazon SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/)
    * [Creating features](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-transform.html)
    * [Creating analysis](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-analyses.html)
    * [Analyzing the data and features](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html)
3. [Submiting the data to Feature Store](#FeatureStore) - Store the features created in [Amazon SageMaker Feature Store](https://aws.amazon.com/sagemaker/feature-store/)
  
---

## Background

In the previous [Introduction lab](../0-Introduction/introduction.ipynb) we created a S3 bucket and uploaded the raw data to it.

Let's get started!

Get variables from previous configuration notebook:

In [ ]:
%store -r bucket
%store -r region
%store -r prefix
%store -r s3uri_raw
%store -r docker_image_name
%store -r framework_version
%store -r athena_table_name
bucket, prefix, s3uri_raw, region, docker_image_name, framework_version, athena_table_name

Let's import the libraries for this lab:

In [ ]:
import sagemaker

sess = sagemaker.Session()
#bucket = sess.default_bucket()
#prefix = "sagemaker/DEMO-xgboost-churn"

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()
role

In [ ]:
import io
import os
import sys
import time
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display as dis
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from IPython import display 

# Importing Data on DataWrangler

To start, we will create a new flow and import the raw data to perform analysis and transformations on it. On the left menu, click on "Home", select "Data Wrangler" on the "Data" section:

<img src="media/011-dw-home.png" width="50%" />

On Data Wrangler Home, click on "Import Data":

<img src="media/012-dw-home-2.png" width="50%" />

As soon as we click on "Import Data", Data Wrangler will be on a loading state. After a couple minutes you should be able to import the raw data on it. While we wait, we can rename our flow by right clicking on the flow tab and choosing "Rename Data Wrangler Flow...".

<img src="media/020-load_new_flow.png" width="100%" />

Let's call the file `churn.flow`

After Data Wrangler finishes loading, we may proceed importing our data. We'll be importing our data from Amazon Athena. The following images guide us through the process.

<img src="media/015-start.png" width="80%" />

After select Athena: 
* Step1: Name your connection
* Step2: Click on "Connect"

<img src="media/016-connection.png" width="100%" />

##### Copy your Athena Table Name

In [ ]:
athena_table_name

After connect on Athena, run following steps:
1. Select AwsGlueCatalog as Data Catalog
2. Select sagemaker_featurestore as Database
3. Copy following block, replacing **{{athena_table_name}}** for your actual table name:

```sql
SELECT state, acc_len, area_code, phone, intl_plan, vmail_plan, 
       vmail_msg, day_mins, day_calls, day_charge, eve_mins, 
       eve_calls, eve_charge, night_mins, night_calls, night_charge, 
       intl_mins, intl_calls, intl_charge, cust_serev_calls, churn
FROM "{{athena_table_name}}" 
```

4. Click on Run (to preview Table)
5. Click on "Import Query"

<img src="media/017-athena-query.png" width="100%" />

After this, name your dataset:

<img src="media/018-athena-name.png" width="80%" />

# Data Analysis on Data Wrangler

Next, we are going to analyze our data by understanding feature distribution and how each of them impacts our target column. Data Wrangler allow us to perform such analysis inside user interface, let's start creating these analysis.

To start our analysis, we'll create a summary of our imported data. The summary can be rertieved by adding a new analysis on Data Wrangler. The following images show the step by step to create our table summary.

<img src="media/061-add-analysis.png" width="100%" />

<img src="media/071-table-summary.png" width="100%" />

By modern standards, it’s a relatively small dataset, with only 3,333 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State` (`state`): the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length` (`acc_len`): the number of days that this account has been active
- `Area Code` (`area_code`): the three-digit area code of the corresponding customer’s phone number
- `Phone` (`phone`): the remaining seven-digit phone number
- `Int’l Plan` (`intl_plan`): whether the customer has an international calling plan: yes/no
- `VMail Plan` (`vmail_plan`): whether the customer has a voice mail feature: yes/no
- `VMail Message` (`vmail_msg`): presumably the average number of voice mail messages per month
- `Day Mins` (`day_mins`): the total number of calling minutes used during the day
- `Day Calls` (`day_calls`): the total number of calls placed during the day
- `Day Charge` (`day_charge`): the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls` (`cust_serev_calls`): the number of calls placed to Customer Service
- `Churn?` (`churn`): whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Continuing our data analysis, we can leverage different Analysis type on Data Wrangler. Next, we can use Histogram to understand our feature distribution and how it impacts our target value. Going back to the "Analyze" tab, we can add a new Analysis. Then, we can choose Histogram as the "Analysis type" and select a feature to explore its distribution. On the following images we've chosen `Account Length` as feature and colored it by our target variable `Churn?`.

<img src="media/091-analysis-tab.png" width="100%" />

<img src="media/101-acc-len-hist.png" width="100%" />

As we can see, we are able to check the distribution of our feature and how its distribution relates to our target value. Feel free to create new histograms for any other feature!

For our next analysis, we'll leverage the "Quick Model" analysis provided by Data Wrangler. This analysis trains a Random Forest algorithm on its own and calculates a feature importance score for each feature on our dataset. You can learn more about the "Quick Model" analysis on this [page](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-analyses.html#data-wrangler-quick-model) of the Amazon SageMaker Data Wrangler documentation.


<img src="media/111-quick-model.png" width="100%" />

The higher the score, the more important the feature is. Therefore, feature `Day Mins` is the most important feature on our dataset according to the "Quick Model" analysis.

Lastly, let's create a complete report about your dataset. This can help you during your data engineering process. More information can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-data-insights.html).

It can be done by clicking on "Get data insights" on your flow screen or by creating a "New Analysis" and choosing "Data Quality and Insights Report". 

<img src="media/112-flow.png" width="100%" />

<img src="media/113-dq-report.png" width="100%" />

<img src="media/114-dq-report-2.png" width="100%" />

# Data Transforms on Data Wrangler

Go back to the DAG view by clicking on `data flow` tab (on the top  left).

Follow the instructions on the image below:  
1 - Click on the plus (+) button  
2 - Click Edit

<img src="media/121-edit.png" width="100%" />

3 - Find the column you want to change  
4 - Select the desired type from the dropdown (change **area_code** to String)   
5 - Click **preview**  
6 - Click **apply**

<img src="media/131-area-code.png" width="100%" />

Once you finish, click "data flow" on the top right corner (to return to dag view)

### Now lets drop the Phone column by adding a Transform  
1 - Click the plus (+)  
2 - Add Transform

<img src="media/141-add-transf.png" width="100%" />

3 - Click on add step

<img src="media/142-add-step.png" width="100%" />

4 - Select the option "Manage Columns"

<img src="media/143-man-col.png" width="100%" />

5 - On dropdown, keep "drop column" option  
4 - Select the `Phone` column from the dropdown (as shown in step 2)  
5 - Click on preview  
6 - Click apply

<img src="media/151-steps-drop.png" width="100%" />

### Now lets Drop a few more columns  
I'll be dropping the first (Day Charge) as an example, just repeat the steps in the image below for the following columns:
* "day_charge"
* "eve_charge"
* "night_charge"
* "intl_charge"

**Hint**: It's possible to drop all at the same step.

<img src="media/171-drop-4.png" width="100%" />

## Now lets Run Some Custom Transformations
### Firstly, change order of the columns 
You can copy the code for the custom transform here:
```python
df = df[['state','acc_len','area_code', 'intl_plan','vmail_plan',
               'vmail_msg','day_mins','day_calls','eve_mins','eve_calls',
               'night_mins','night_calls','intl_mins','intl_calls',
               'cust_serev_calls','churn']]
```

Select "Custom Transform" option:

<img src="media/181-custom-1.png" width="100%" />

<img src="media/184-reordering.png" width="100%" />

### Now lets rename all columns using a custom transform
You can copy the code for the custom transform here:
```python
df = df.rename(columns= 
                 {'state': 'State' , 
                  'acc_len': 'Account Length' ,
                  'area_code': 'Area Code' ,
                  'intl_plan': "Int'l Plan" ,
                  'vmail_plan': 'VMail Plan' ,
                  'vmail_msg': 'VMail Message' ,
                  'day_mins': 'Day Mins' ,
                  'day_calls': 'Day Calls' ,
                  'eve_mins': 'Eve Mins' ,
                  'eve_calls': 'Eve Calls' ,
                  'night_mins': 'Night Mins' ,
                  'night_calls': 'Night Calls' ,
                  'intl_mins': 'Intl Mins' ,
                  'intl_calls': 'Intl Calls' ,
                  'cust_serev_calls': 'CustServ Calls' ,
                  'churn': 'Churn?'
                  })
```

<img src="media/185-rename.png" width="100%" />

### Now lets do another custom transform for OneHot Encoding
You can copy the code for the custom transform here:
```python
import pandas as pd

model_data = pd.get_dummies(df)

df = pd.concat(
  [model_data["Churn?_True."],
   model_data.drop( ["Churn?_False.", "Churn?_True."], axis=1)],
  axis=1
).rename( 
  columns = {
    "Churn?_True.": "Churn"
  }
)
```

<img src="media/181-custom-1.png" width="100%" />

<img src="media/182-custom-2.png" width="100%" />

# Exporting Transformed data on Data Wrangler

After performing the transformations needed on our dataset, we'll export the transformed data to our S3 bucket. We are able to do so inside Data Wrangler UI by following the steps highlighted on the images below.

- Return to Data Wrangler Flow
- Click on "Export to"
- Pick "Amazon S3" option

<img src="media/191-export.png" width="100%" />

After selecting the `Save to S3` option, a new notebook will be displayed similar to the one presented on the image below.

<img src="media/210-export_to_s3_notebook.png" width="100%" />

We can proceed to `Run all cells`, as descrribed in the notebook. 

The processing job will start and it will take a few minutes to finish. 

Upon completion, we'll see a similar output on the cell presented on the following image.

<img src="media/219-dw-notebook-run-all.png" width="100%" />

In the end of that notebook check that Processing Job is running (with the Data Wrangler Docker image):

<img src="media/220-processing_job_finished.png" width="100%" />

On the left menu, click on "Home" icon and select "Experiments":

<img src="media/226-experiments.png" width="60%"/>

Select "Unassigned runs" and click on Last executed Processing Job:

<img src="media/227-runs.png" width="70%"/>

Click on "Output Artifacts" on Left Menu and copy S3 URI output value

<img src="media/228-outputs.png" width="100%"/>

<h3 style="color: red;">Important  </h3>

Paste your S3 URI below:

In [ ]:
s3uri_processed = "s3://<your-bucket>/export-flow-01-15-11-32-3a9d8022/output"

In [ ]:
s3uri_processed_file = sagemaker.s3.S3Downloader.list(s3uri_processed)[0]
s3uri_processed_file

If you want to check it in S3 console run the cell bellow and click the link:

In [ ]:
from IPython.core.display import display, HTML
from sagemaker.s3 import parse_s3_url

out_bucket, out_prefix = parse_s3_url(s3uri_processed_file)
out_path = os.path.dirname(out_prefix)
out_file = os.path.basename(out_prefix)

s3_url_placeholder = "https://s3.console.aws.amazon.com/s3/buckets/{}?&prefix={}/"
display(HTML(f"<a href={s3_url_placeholder.format(out_bucket, out_path)}>Go to S3 console and check output of Data Wrangler</a>"))

In the S3 console you should see:

<img src="media/230-download_transformed_data_s3.png" width="100%" />

(If you want to download the data to you computer follow the steps in the image above)

Let's download the data to Studio:

In [ ]:
sess.download_data(".", 
                   out_bucket, 
                   key_prefix=out_prefix)

Click the refresh button on Studio. You should see something like:

<img src="media/230-download_transformed_data_s3_local.png" width="50%" />

(The CSV file is downloaded)

In [ ]:
out_file

In [ ]:
model_data = pd.read_csv(out_file)
model_data.head()

Above we should see the transformed data with `Churn` in the first columns, the one-hot-encoded columns and so on.

Finally, let's break the data into **train, validation and test sets:**

In [ ]:
train_data, validation_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
)
train_data.shape, validation_data.shape, test_data.shape

In [ ]:
train_data.head(2)

In [ ]:
model_data.shape

Create CSV files for the 3 datasets:

In [ ]:
train_file_name = "train.csv"
validation_file_name = "validation.csv"
test_file_name = "test.csv"

train_data.to_csv(train_file_name , header=False, index=False)
validation_data.to_csv(validation_file_name, header=False, index=False)
test_data.to_csv(test_file_name, header=False, index=False)

Lastly, we'll upload these files to S3.

In [ ]:
# Return the URLs of the uploaded files, so they can be reviewed or used elsewhere
train_dir = f"{prefix}/data/train"
val_dir = f"{prefix}/data/validation"
test_dir = f"{prefix}/data/test"

s3uri_train = sagemaker.s3.S3Uploader.upload(train_file_name, f's3://{bucket}/{train_dir}')
s3uri_validation = sagemaker.s3.S3Uploader.upload(validation_file_name, f's3://{bucket}/{val_dir}')
s3uri_test = sagemaker.s3.S3Uploader.upload(test_file_name, f's3://{bucket}/{test_dir}')
s3uri_train, s3uri_validation, s3uri_test

Save the S3 URIs for the 3 datasets for later:

In [ ]:
%store s3uri_train
%store s3uri_validation
%store s3uri_test